# Fine-tuning Deep Learning models
When possible, it is a good idea to fine-tune existing CNN that have been pre-trained on large dataset. This can be done when the dataset under consideration is similar in context to the dataset that has been used to pre-train the model. We can then used a state of the art CNN that has already learned features that are relevant to our own deep learning task. We are then only left with calculating the bottleneck features of the pre-trained CNN and fine-tuned the pre-trained weights.

In the following, we use the ResNet50 deep learning model. The model is available in keras along with its weights pre-trained on the [ImageNet](http://www.image-net.org/) dataset, a collection of 1.2M labeled images with the presence of 1000 object categories.